I will build an Instagram bot that automatically posts new artist books and zines collected from a list of New York independent bookstores. 
Steps:
1. web scraping from sites like Printed Matter, Center for Book Arts, Bungee Space, Bluestockings, Village Works, Climax Books, and Quimby's, compiling the results into a unified .csv file hosted on github. 
2. Detect and append newly added entries/titles from these sites to the existing dataset to update it (need a manual trigger)
3. Convert the data into post-ready text and images
4. connecting to Instagram via the Instagram Graph API
5. Schedule and automate posts using Python (with the instagram Graph API and instabot library)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
#Scrape CBA's artists books from their bookshop page
headers = {
"User-Agent": "Mozilla/5.0"
}

all_books = []

for page in range(1, 8):
    url = f"https://centerforbookarts.org/book-shop?cats=artists-books&pg={page}"
    response = requests.get(url, headers=headers)

    soup = BeautifulSoup(response.content, 'html.parser')
    posts = soup.find_all('div', class_='post-content')

    for post in posts:
        category_div = post.find('div', class_='post-header')
        category = category_div.get_text(strip=True) if category_div else None

        title_tag = post.find('h2', class_='post-title')
        title = title_tag.get_text(strip=True) if title_tag else None

        image_div = post.find('div', class_='post-image')
        img_tag = image_div.find('img', decoding='async') if image_div else None
        image_link = img_tag['src'] if img_tag else None

        a_tag = post.find('a') if post else None
        book_url = a_tag['href'] if a_tag and 'href' in a_tag.attrs else None

        all_books.append({
            'category': category,
            'title': title,
            'image link': image_link,
            'url': book_url
        })

    time.sleep(1) 

# Save to CSV
CBA = pd.DataFrame(all_books)
CBA

""


In [5]:
soup

<!DOCTYPE html>
<html lang="en-US"><head><title>Just a moment...</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=Edge" http-equiv="X-UA-Compatible"/><meta content="noindex,nofollow" name="robots"/><meta content="width=device-width,initial-scale=1" name="viewport"/><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;backgro

In [6]:
#Scrape the recent books from Bungee Space's bookshop page



url = f"https://3ssstudios.com/collections/publications"
headers = {
"User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

catalog_container = soup.find("div", class_="grid grid--uniform grid--view-items")

items = []

for item in catalog_container.find_all("div", class_="grid-view-item"):
    img_tag = item.find("img", class_="grid-view-item__image")
    image_link = img_tag['src'] if img_tag and 'src' in img_tag.attrs else None

    title_tag = item.find("div", class_="h4 grid-view-item__title")
    title = title_tag.get_text(strip=True) if title_tag else None

    a_tag = item.find("a", href=True)
    url = f"https://3ssstudios.com/{a_tag['href']}" if a_tag else None

    items.append({
        "title": title,
        "image link": image_link,
        "url": url
    })



df = pd.DataFrame(items)
df


,title,image link,url
0,Serial Killers with Head Injuries,//3ssstudios.com/cdn/shop/files/1_094d19c1-e2d...,https://3ssstudios.com//collections/publicatio...
1,10 Signs You Live in a Haunted House,//3ssstudios.com/cdn/shop/files/1_44db0c67-9c4...,https://3ssstudios.com//collections/publicatio...
2,Everybody Should Have Busy: 2 Weeks of Signs a...,//3ssstudios.com/cdn/shop/files/0_76416f88-299...,https://3ssstudios.com//collections/publicatio...
3,An Incomplete Radical Feminist Education from ...,//3ssstudios.com/cdn/shop/files/5_1b1e3836-859...,https://3ssstudios.com//collections/publicatio...
4,The Words of a Gentleman,//3ssstudios.com/cdn/shop/files/0_ab817cdc-6f3...,https://3ssstudios.com//collections/publicatio...
5,Chutney Magazine #4,//3ssstudios.com/cdn/shop/files/1_c5d69349-4d6...,https://3ssstudios.com//collections/publicatio...
6,TREES OF AMERICA,//3ssstudios.com/cdn/shop/files/1_3bab5af7-3f1...,https://3ssstudios.com//collections/publicatio...
7,THE BOX SET,//3ssstudios.com/cdn/shop/files/set1_1080x1530...,https://3ssstudios.com//collections/publicatio...
8,Railing Codex,//3ssstudios.com/cdn/shop/files/1_b4d6afb5-f03...,https://3ssstudios.com//collections/publicatio...
9,PLATES Iteration 2: Body Issue,//3ssstudios.com/cdn/shop/files/1_73f16a10-b0a...,https://3ssstudios.com//collections/publicatio...


In [7]:
df.to_csv("bungee_space_books.csv", index=False)